In [29]:
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [31]:
df_train = pd.read_csv("/content/drive/MyDrive/Sign_Language/sign_mnist_train.csv")

In [32]:
# Separate labels and pixel values
labels = df_train.iloc[:, 0].values
pixels = df_train.iloc[:, 1:].values
print(f"labels = {labels}\npixels = {pixels}")

labels = [ 3  6  2 ... 18 17 23]
pixels = [[107 118 127 ... 204 203 202]
 [155 157 156 ... 103 135 149]
 [187 188 188 ... 195 194 195]
 ...
 [174 174 174 ... 202 200 200]
 [177 181 184 ...  64  87  93]
 [179 180 180 ... 205 209 215]]


In [25]:
df_train

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [101]:
class AlexNet(nn.Module):
  def __init__(self, num_classes, dropout=0.5):
      super(AlexNet, self).__init__()
      self.features = nn.Sequential(
          nn.Conv2d(1, 64, 5, stride=1, padding=2),  # Adjusted kernel size, stride, and padding
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),  # Adjusted kernel size and stride
          nn.Conv2d(64, 192, 5, stride=1, padding=2),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),
          nn.Conv2d(192, 384, 3, stride=1, padding=1),
          nn.ReLU(),
          nn.Conv2d(384, 256, 3, stride=1, padding=1),
          nn.ReLU(),
          nn.Conv2d(256, 256, 3, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2)  # Adjusted kernel size and stride
      )
      self.classifier = nn.Sequential(
          nn.Dropout(p=dropout),
          nn.Linear(256 * 3 * 3, 4096),  # Adjusted dimensions based on feature map size
          nn.ReLU(),
          nn.Dropout(p=dropout),
          nn.Linear(4096, 4096),
          nn.ReLU(),
          nn.Linear(4096, num_classes),  # Adjusted to match the number of classes
          nn.Softmax(dim=1)
      )

  def forward(self, x):
      x = self.features(x)
      x = x.view(x.size(0), -1)  # Flatten the tensor
      x = self.classifier(x)
      return x

In [112]:
num_classes = df_train.iloc[:, 0].nunique() + 1 # starts from 1
print(f'Number of unique labels: {num_classes}')

Number of unique labels: 25


In [113]:
# Define your device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AlexNet(num_classes=num_classes).to(device)
epochs = 2 # number of single passes on the network
loss_fnPrecents = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [104]:
class SignDataset(Dataset):
    def __init__(self, pixels, labels, transform=None):
        self.pixels = pixels
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.pixels[idx].reshape(28, 28).astype('float32')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [117]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = SignDataset(pixels, labels, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True)

In [119]:
# Training loop
def train_model(model, dataloader, epochs, optimizer, loss_fn, device):
    model.train()
    for epoch in range(epochs):
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = loss_fn(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

def evaluate_model(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    total, correct = 0, 0
    with torch.no_grad():
        for data, targets in dataloader:
            data, targets = data.to(device), targets.to(device)  # Ensure data and targets are on the correct device
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    return 100 * correct / total  # Return accuracy as a percentage

In [118]:
train_model(model=model,
            dataloader=train_dataloader,
            epochs=epochs,
            optimizer=optimizer,
            loss_fn=loss_fnPrecents,
            device=device)

KeyboardInterrupt: 